In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import DBSCAN, KMeans

In [2]:
location = "okcupid_profiles.csv"
data = pd.read_csv(location)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59946 entries, 0 to 59945
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          59946 non-null  int64  
 1   status       59946 non-null  object 
 2   sex          59946 non-null  object 
 3   orientation  59946 non-null  object 
 4   body_type    54650 non-null  object 
 5   diet         35551 non-null  object 
 6   drinks       56961 non-null  object 
 7   drugs        45866 non-null  object 
 8   education    53318 non-null  object 
 9   ethnicity    54266 non-null  object 
 10  height       59943 non-null  float64
 11  income       59946 non-null  int64  
 12  job          51748 non-null  object 
 13  last_online  59946 non-null  object 
 14  location     59946 non-null  object 
 15  offspring    24385 non-null  object 
 16  pets         40025 non-null  object 
 17  religion     39720 non-null  object 
 18  sign         48890 non-null  object 
 19  smok

Variables numericas: age, height, income
Variables que sacamos por ahora: last_online, todos los essays
Variables con nulls=

In [4]:
#Para las variables categóricas
def completar_nodijo(columna):
    mask_null = columna.isnull()
    columna[mask_null] = "rather not say"
    

In [5]:
columnas_cat_a_modificar = ['status', 'sex', 'orientation', 'body_type', 'diet', 'drinks',
       'drugs', 'education', 'ethnicity','job', 'location', 'offspring', 'pets', 'religion', 'sign',
       'smokes', 'speaks']

In [6]:
for i in columnas_cat_a_modificar:
    completar_nodijo(data[i])

/tmp/ipykernel_921960/1023053484.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  columna[mask_null] = "rather not say"


In [7]:
data.drugs.value_counts()

never             37724
rather not say    14080
sometimes          7732
often               410
Name: drugs, dtype: int64

In [8]:
#Columnas que no vamos a utilizar ahora
columnas_no_utilizadas = ['essay0', 'essay1', 'essay2', 'essay3', 'essay4',
       'essay5', 'essay6', 'essay7', 'essay8', 'essay9']

In [9]:
data_essays = data[columnas_no_utilizadas]

In [10]:
data = data.drop(columnas_no_utilizadas, axis=1)

In [11]:
data.height[data.height.isnull()] = -1
#Seria más prolijo hacerlo con un fill na

/tmp/ipykernel_921960/3026053226.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.height[data.height.isnull()] = -1


In [12]:
print(f"{(data.income != -1).sum()} datos de income valido de {len(data.income)}")

11504 datos de income valido de 59946


In [13]:
print(f"{(data.age != -1).sum()} datos de age valido de {len(data.age)}")

59946 datos de age valido de 59946


In [14]:
data_last_online = data.pop("last_online")

## **Vamos a separar el dataset en variables categóricas y nominales**
Revisar después cuando hagan el laburo en fino de las columnas

In [15]:
data_cat = data[columnas_cat_a_modificar]
data_cat_speaks = data_cat.pop("speaks")
data_cat_ethnicity = data_cat.pop("ethnicity")
data_cat_sign = data_cat.pop("sign")
data_cat_religion = data_cat.pop("religion")
data_nom = data[["age", "height", "income"]]

# Separación por etnia

In [16]:
#obtener un conjunto unico labels por cada etnicity 
#data["ethnicity"].value_counts()
enthinicity_labels = set()

#quiero lista unica de elementos x eso uso set()
for label in data["ethnicity"]:
    etnias = label.split(',')
    # para que no se repita el nombre si o si debo remover espacios
    for i in etnias:
        etnia = i.strip()

        enthinicity_labels.add(etnia)
    
# enthinicity_labels.remove('rather not say') ?    para trabajar c lista
enthinicity_labels = list(enthinicity_labels)
enthinicity_labels 

['white',
 'other',
 'hispanic / latin',
 'rather not say',
 'middle eastern',
 'native american',
 'asian',
 'pacific islander',
 'indian',
 'black']

In [17]:
# Obtener columnas indicadoras por cada valiable categorica 

total_puntos = len(data["ethnicity"])


ethni_cols = {}   

for etnia in enthinicity_labels:
    ethni_cols[etnia] = [0.0 for i in range(total_puntos)]
    
for indice in range(total_puntos):
    etnias = data["ethnicity"][indice]
    for etnia in etnias.split(','):
        ethni_cols[etnia.strip()][indice] = 1.0
    
# add to DF
data_cat = pd.merge(data_cat, pd.DataFrame(ethni_cols),how="inner", left_index=True, right_index=True)

Separación por locación

In [18]:
#separando en columnas los elementos separados por comas

#   separar por comas, primer elemento columna 1 segundo columna dos. diccionario con dos key, valors van a ser lista 1 y lista 2

locs1 = []
locs2 = []
locscountry = []

# crear diccionario clave valor columna lista
# 

for label in data["location"]:
    locs = label.split(',')

    #valido si tengo una coma inesperada
    
    if len(locs) > 2:
        locscountry.append(locs[2])
    else:
        locscountry.append("US")

    locs1.append(locs[0])
    locs2.append(locs[1])
location_split = {'city' : locs1 , 'state' : locs2 , 'country' : locscountry}

df_location = pd.DataFrame(location_split)
df_location

# No lo usamos porque estamos usando coordenadas con datos nominales, dejo comentado:
# data_cat = pd.merge(data_cat, df_location, how="inner", left_index=True, right_index=True)

,city,state,country
0,south san francisco,california,US
1,oakland,california,US
2,san francisco,california,US
3,berkeley,california,US
4,san francisco,california,US
...,...,...,...
59941,oakland,california,US
59942,san francisco,california,US
59943,south san francisco,california,US
59944,san francisco,california,US


In [19]:
# Esto no funciona pq es super lento ir uno por uno:
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="locator")

# latitude = []
# longitude = []

# for label in data["location"]:
#     loc = geolocator.geocode(label)
#     if loc is not None:
#         latitude.append(loc.latitude)
#         longitude.append(loc.longitude)
#     else:
#         print(f"Warning: {label} not recognised")
#         latitude.append(float("nan"))
#         longitude.append(float("nan"))

# location_coords = {'latitude' : latitude , 'longitude' : longitude}

# data_nom += pd.DataFrame(location_coords)

In [20]:
# convertir locacion a coordendas es super lento porque geopy llama a un servicio
# para evitar repeticiones primero reducimos con un dict

lugares = {}
for label in data["location"]:
    lugares[label.strip()] = ()

puntos = len(data["location"])
print(f"Encontrados {len(lugares)} en {puntos} puntos de dato")

# busca las coordenadas

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="locator")

for lugar, coords in lugares.items():
    loc = geolocator.geocode(lugar)
    if loc is not None:
        lugares[lugar] = (loc.latitude, loc.longitude)
    else:
        print(f"Lugar no encontrado: {lugar}")
    


Encontrados 199 en 59946 puntos de dato
Lugar no encontrado: green brae, california


In [21]:

lugares["green brae, california"] = (38.4257453,-121.5963564) # agrego Greenbrae manual

latitude = []
longitude = []

for label in data["location"]:
    latitude.append(lugares[label.strip()][0])
    longitude.append(lugares[label.strip()][1])

location_coords = {'latitude' : latitude , 'longitude' : longitude}

data_nom = pd.merge(data_nom, pd.DataFrame(location_coords), how="inner", left_index=True, right_index=True)

Separación por lenguages

In [22]:
# aca removemos la aclaracion de sabiduria pq sino explota el tamanio
def clean_language(language):
    # eliminamos parentesis que aclara conocimiento, sino el dataset explota
    # aca podiramos usar regex pero find es mas simple
    par_find = language.find(' (') 
    if par_find >= 0:
        language = language[0:par_find]
    return language.strip()


#obtener un conjunto unico labels por cada lenguaje 
language_labels = set()

#quiero lista unica de elementos x eso uso set()
for label in data["speaks"]:
    languages = label.split(',')
    # para que no se repita el nombre si o si debo remover espacios
    for language in languages:
        language_labels.add(clean_language(language))
    
# language_labels.remove('rather not say') ?    para trabajar c lista


# Obtener columnas indicadoras por cada valiable categorica 

total_puntos = len(data["speaks"])


language_cols = {}   

for language in language_labels:
    language_cols[language] = [0.0 for i in range(total_puntos)]
    
for indice in range(total_puntos):
    languages = data["speaks"][indice]
    for language in languages.split(','):
        language_cols[clean_language(language)][indice] = 1.0
    
# add to DF
data_lan = pd.DataFrame(language_cols)

In [23]:
language_labels

{'afrikaans',
 'albanian',
 'ancient greek',
 'arabic',
 'armenian',
 'basque',
 'belarusan',
 'bengali',
 'breton',
 'bulgarian',
 'c++',
 'catalan',
 'cebuano',
 'chechen',
 'chinese',
 'croatian',
 'czech',
 'danish',
 'dutch',
 'english',
 'esperanto',
 'estonian',
 'farsi',
 'finnish',
 'french',
 'frisian',
 'georgian',
 'german',
 'greek',
 'gujarati',
 'hawaiian',
 'hebrew',
 'hindi',
 'hungarian',
 'icelandic',
 'ilongo',
 'indonesian',
 'irish',
 'italian',
 'japanese',
 'khmer',
 'korean',
 'latin',
 'latvian',
 'lisp',
 'lithuanian',
 'malay',
 'maori',
 'mongolian',
 'norwegian',
 'occitan',
 'other',
 'persian',
 'polish',
 'portuguese',
 'rather not say',
 'romanian',
 'rotuman',
 'russian',
 'sanskrit',
 'sardinian',
 'serbian',
 'sign language',
 'slovak',
 'slovenian',
 'spanish',
 'swahili',
 'swedish',
 'tagalog',
 'tamil',
 'thai',
 'tibetan',
 'turkish',
 'ukrainian',
 'urdu',
 'vietnamese',
 'welsh',
 'yiddish'}

Separación por signo

In [24]:
#separando en columnas. el primer elemento es el signo, y el segundo, cuanto le importa

sign = []
sign_matters = []

# crear diccionario clave valor columna lista


for label in data["sign"]:
    signs = label.split(maxsplit=1)

    sign.append(signs[0])
    if len(signs)>1:
        sign_matters.append(signs[1])
    else:
        sign_matters.append('rather not say')  
    
df_sign = pd.DataFrame({'sign' : sign , 'sign_matters' : sign_matters})
data_cat = pd.merge(data_cat, df_sign, how="inner", left_index=True, right_index=True)

Separación por pets

In [25]:
#separando en columnas. el primer elemento es el signo, y el segundo, cuanto le importa

dogs = []
cats = []

# crear diccionario clave valor columna lista


for label in data["pets"]:
    
    # Modifico la columna para que tenga toda el mismo formato
    # Cambio las que no tenían dato
    if label == 'rather not say':
        label = 'no_opinion dogs and no_opinion cats'
    # Cambio las que tienen opinión solo de una mascota
    if label.find('and') == -1:
        if label.find('cats') == -1:
            label = label + " and no_opinion cats"
        else:
            label = "no_opinion dogs and " + label
    
    pets = label.split('and')

    dogs.append(pets[0])
    cats.append(pets[1])

    
df_pets = pd.DataFrame({'dogs' : dogs , 'cats' : cats})
data_cat = pd.merge(data_cat, df_pets, how="inner", left_index=True, right_index=True)

In [26]:
# Save dataframes

data_nom.to_csv("data_nom.csv", index = False)
data_cat.to_csv("data_cat.csv", index = False)
data_lan.to_csv("data_lan.csv", index = False)